# Importando bibliotecas

In [1]:
import numpy as np         # para fazer contas e mexer com matrizes
import pandas as pd        # para montar DataFrames (tabelas de bancos de dados)

import matplotlib.pyplot as plt
import seaborn as sns

import modelos as m

# CUIDADO!!! Para esconder os avisos do curvefit():
import warnings
warnings.filterwarnings("ignore")

In [2]:
# para não precisar reiniciar o kernel do jupyter-notebook quando mudar os arquivos .py
%load_ext autoreload
%autoreload 2

In [3]:
sns.set_theme()

# Lendo dados

In [4]:
resultdf_indiv = pd.read_pickle('./output/dados_eit_modelos.pkl')
resultdf_indiv.tail(2)

,Animal,Maneuver,n_steps,Pressures,Volumes,Model,Method,TLC_index,N_points_interp,p0,bounds,fitted,parameters,Vol_CT,error,fit error,error factor,Raw data
1798,mra33,B,3,"[10.303646087646484, 14.871094226837158, 20.07...","[146.587890625, 330.3841552734375, 560.5339355...",sigmoidmurphyoffset,dogbox,0,20,"[2000.0, -200.0, 6000, 6000, 32.0]","([500, -600, 2000, 2000, 10], [3500, -100, 200...",True,"[2399.162461735464, -600.0, 20000.0, 12249.219...",2154.6,11.350713,0.149489,3.655387,"[[10.303646087646484, 146.587890625], [20.1421..."
1799,mra33,C,4,"[10.202864646911621, 15.145573139190674, 20.05...","[-56.06181335449219, 113.50151824951172, 334.8...",sigmoidmurphyoffset,dogbox,0,20,"[2000.0, -200.0, 6000, 6000, 32.0]","([500, -600, 2000, 2000, 10], [3500, -100, 200...",True,"[2287.6878993115215, -599.9999999976203, 20000...",2154.6,6.176919,0.064135,1.364676,"[[10.202864646911621, -56.06181335449219], [20..."


In [5]:
dados_ct_df = pd.read_pickle('./volume_referencia/dados_ct_completo.pkl')
dados_ct_df.head(2)

,Animal,Pressure,CT Volume,Volume (Luva),Estimated TLC,Estimated K,CT_Volume (rec),Pressure (rec),CT folders,CT folders (rec),Recruited percentage,Recruitment potential
0,mra26,"[10, 30, 45]","[458.40117130279543, 1406.1211183943176, 1969....","[1367.135, 1944.058, 2571.724]",2622.866589,0.030899,"[0, 1373.4339242285157, 1969.8752383276367]","[0, 24, 45]",[/mnt/hdsda3/arquivos/Recrutamento/CT_data_org...,[/mnt/hdsda3/arquivos/Recrutamento/CT_data_org...,"[0.657497323175345, 0.8872168925301633, 1.0]","[0.34250267682465496, 0.11278310746983666, 0.0]"
1,mra28,"[10, 20, 25, 30, 45]","[539.5733922408295, 866.4519947195436, 1110.76...","[1283.233, 1660.177, 1847.163, 2051.841, 2076....",2105.897077,0.044662,"[0, 1384.9271159648897, 1823.6774616718296]","[0, 24, 45]",[/mnt/hdsda3/arquivos/Recrutamento/CT_data_org...,[/mnt/hdsda3/arquivos/Recrutamento/CT_data_org...,"[0.7112959429745049, 0.6965586330935198, 0.784...","[0.28870405702549506, 0.30344136690648016, 0.2..."


In [6]:
def pega_potencial(df):
    dfresult_lst = []
    for index, ser in df.iterrows():
        animal = ser['Animal']
        manobra = {
            10: 'A',
            20: 'B',
            25: 'C',
            30: 'D',
            45: 'R'
        }
        for p,r,rp in zip(ser['Pressure'],ser['Recruited percentage'],ser['Recruitment potential']):
            index = []
            caso = []
            index.append('Animal')
            caso.append(animal)
            index.append('Maneuver')
            caso.append(manobra[p])
            index.append('Pressure')
            caso.append(p)
            index.append('Recruited percentage')
            caso.append(r)
            index.append('Recruitment potential')
            caso.append(rp)
            casodf = pd.DataFrame(caso, index).T
            dfresult_lst.append(casodf)
        
    dfresult = pd.concat(dfresult_lst, ignore_index=True)
    return dfresult

In [7]:
dados_ct_rec_df = pega_potencial(dados_ct_df)
dados_ct_rec_df.tail(3)

,Animal,Maneuver,Pressure,Recruited percentage,Recruitment potential
29,mra34,C,25,0.674682,0.325318
30,mra34,D,30,0.737226,0.262774
31,mra34,R,45,1.0,-0.0


In [8]:
dados_ct_util = dados_ct_rec_df[(dados_ct_rec_df['Maneuver']=='B')|
                                (dados_ct_rec_df['Maneuver']=='C')|
                                (dados_ct_rec_df['Maneuver']=='D')].reset_index(drop=True)
dados_ct_util.tail(3)

,Animal,Maneuver,Pressure,Recruited percentage,Recruitment potential
15,mra34,B,20,0.649508,0.350492
16,mra34,C,25,0.674682,0.325318
17,mra34,D,30,0.737226,0.262774


In [9]:
dados_ct_eit = dados_ct_util.merge(resultdf_indiv,on=('Animal','Maneuver'))

In [10]:
dados_ct_eit.groupby('Animal').describe()

Vol_CT                                                                \
        count    mean           std     min     25%     50%     75%     max   
Animal                                                                        
mra26    90.0  2622.9  4.572950e-13  2622.9  2622.9  2622.9  2622.9  2622.9   
mra28   270.0  2105.9  0.000000e+00  2105.9  2105.9  2105.9  2105.9  2105.9   
mra29   270.0  2205.7  9.111836e-13  2205.7  2205.7  2205.7  2205.7  2205.7   
mra31   270.0  1974.4  0.000000e+00  1974.4  1974.4  1974.4  1974.4  1974.4   
mra32   180.0  1440.2  2.280079e-13  1440.2  1440.2  1440.2  1440.2  1440.2   
mra33   270.0  2154.6  0.000000e+00  2154.6  2154.6  2154.6  2154.6  2154.6   
mra34   270.0  2211.2  9.111836e-13  2211.2  2211.2  2211.2  2211.2  2211.2   

        error                                                                \
        count          mean           std         min        25%        50%   
Animal                                                                        
mra26    90.0 -1.920068e+01  2.004173e+01  -66.892014 -35.060934 -28.525283   
mra28   270.0 -1.423092e+01  3.457028e+01  -56.266351 -39.645350 -10.852101   
mra29   270.0 -2.133318e+01  3.714716e+01 -217.466141 -50.714293  -8.012707   
mra31   270.0 -2.073840e+01  3.099051e+01 -178.181948 -42.399106 -23.437827   
mra32   180.0 -6.115262e+00  3.860863e+01  -65.282600 -31.420391   0.000000   
mra33   270.0  1.468839e+01  3.942174e+01  -56.612992  -1.736738   0.000000   
mra34   270.0  4.308913e+06  4.868241e+07 -174.789470  -9.927683   0.000000   

                                 
              75%           max  
Animal                           
mra26    0.000000  3.344009e+01  
mra28    0.000000  1.841342e+02  
mra29    0.000000  1.066457e+02  
mra31    0.000000  7.726904e+01  
mra32    8.055906  1.430218e+02  
mra33   41.598574  1.325553e+02  
mra34   57.114722  7.749351e+08

# Subsets

In [11]:
dados_ct_eit_fitted = dados_ct_eit[dados_ct_eit.fitted==True].reset_index(drop=True)
dados_ct_eit_fitted.tail(1)

,Animal,Maneuver,Pressure,Recruited percentage,Recruitment potential,n_steps,Pressures,Volumes,Model,Method,...,N_points_interp,p0,bounds,fitted,parameters,Vol_CT,error,fit error,error factor,Raw data
1058,mra34,D,30,0.737226,0.262774,5,"[9.9609375, 15.040364742279053, 20.13463592529...","[-167.6243133544922, 13.06080150604248, 203.19...",sigmoidmurphyoffset,trf,...,20,"[2000.0, -200.0, 6000, 6000, 32.0]","([500, -600, 2000, 2000, 10], [3500, -100, 200...",True,"[2586.0833865259333, -599.9999999999999, 19999...",2211.2,16.953843,1.528923,33.286242,"[[9.9609375, -167.6243133544922], [20.25208377..."


In [12]:
dados_ct_eit_fitted.groupby('Animal').describe()

Vol_CT                                                                \
        count    mean           std     min     25%     50%     75%     max   
Animal                                                                        
mra26    65.0  2622.9  0.000000e+00  2622.9  2622.9  2622.9  2622.9  2622.9   
mra28   206.0  2105.9  4.558551e-13  2105.9  2105.9  2105.9  2105.9  2105.9   
mra29   133.0  2205.7  9.129333e-13  2205.7  2205.7  2205.7  2205.7  2205.7   
mra31   179.0  1974.4  2.280115e-13  1974.4  1974.4  1974.4  1974.4  1974.4   
mra32   133.0  1440.2  2.282333e-13  1440.2  1440.2  1440.2  1440.2  1440.2   
mra33   197.0  2154.6  0.000000e+00  2154.6  2154.6  2154.6  2154.6  2154.6   
mra34   146.0  2211.2  4.563128e-13  2211.2  2211.2  2211.2  2211.2  2211.2   

        error                                                         \
        count       mean        std        min        25%        50%   
Animal                                                                 
mra26    65.0 -24.187219  18.418038 -36.594281 -35.353685 -33.816677   
mra28   206.0 -18.652179  38.540131 -56.266351 -40.863687 -32.988805   
mra29   133.0 -34.919147  34.419827 -63.397361 -54.611491 -44.728630   
mra31   179.0 -28.363033  28.808779 -68.490106 -45.115282 -41.729161   
mra32   133.0  -8.146993  44.223237 -58.347592 -39.555832 -27.244132   
mra33   197.0  20.131297  44.975051 -56.612992 -19.275523   7.361986   
mra34   146.0  18.069519  38.965975 -49.163196 -20.464086  17.063562   

                               
              75%         max  
Animal                         
mra26  -21.084918   33.440085  
mra28   -8.948123  184.134175  
mra29  -27.952940  106.645748  
mra31  -23.076872   77.269044  
mra32   28.068253  143.021802  
mra33   62.009213  132.555347  
mra34   58.285094   58.285094

In [13]:
dados_ct_eit_fitted.groupby(['Model','Animal']).describe()

Vol_CT                                        \
                              count    mean           std     min     25%   
Model                 Animal                                                
sigmoid_recruit_units mra26    15.0  2622.9  4.707082e-13  2622.9  2622.9   
                      mra28    30.0  2105.9  4.625214e-13  2105.9  2105.9   
                      mra29    31.0  2205.7  4.622643e-13  2205.7  2205.7   
                      mra31    35.0  1974.4  0.000000e+00  1974.4  1974.4   
                      mra32    25.0  1440.2  0.000000e+00  1440.2  1440.2   
                      mra33    35.0  2154.6  4.613863e-13  2154.6  2154.6   
                      mra34    30.0  2211.2  4.625214e-13  2211.2  2211.2   
sigmoidmurphy         mra26     7.0  2622.9  0.000000e+00  2622.9  2622.9   
                      mra28    30.0  2105.9  4.625214e-13  2105.9  2105.9   
                      mra29     1.0  2205.7           NaN  2205.7  2205.7   
                      mra31    20.0  1974.4  2.332805e-13  1974.4  1974.4   
                      mra32    20.0  1440.2  2.332805e-13  1440.2  1440.2   
                      mra33    30.0  2154.6  4.625214e-13  2154.6  2154.6   
                      mra34    16.0  2211.2  0.000000e+00  2211.2  2211.2   
sigmoidmurphyoffset   mra26     7.0  2622.9  0.000000e+00  2622.9  2622.9   
                      mra28    31.0  2105.9  4.622643e-13  2105.9  2105.9   
                      mra29     1.0  2205.7           NaN  2205.7  2205.7   
                      mra31    23.0  1974.4  4.649677e-13  1974.4  1974.4   
                      mra32    20.0  1440.2  2.332805e-13  1440.2  1440.2   
                      mra33    32.0  2154.6  0.000000e+00  2154.6  2154.6   
                      mra34    16.0  2211.2  0.000000e+00  2211.2  2211.2   
sigmoidpaivaoffset    mra26    15.0  2622.9  4.707082e-13  2622.9  2622.9   
                      mra28    44.0  2105.9  4.600047e-13  2105.9  2105.9   
                      mra29    44.0  2205.7  0.000000e+00  2205.7  2205.7   
                      mra31    40.0  1974.4  0.000000e+00  1974.4  1974.4   
                      mra32    28.0  1440.2  4.630920e-13  1440.2  1440.2   
                      mra33    40.0  2154.6  0.000000e+00  2154.6  2154.6   
                      mra34    30.0  2211.2  4.625214e-13  2211.2  2211.2   
sigmoidvenegas1       mra26    15.0  2622.9  4.707082e-13  2622.9  2622.9   
                      mra28    44.0  2105.9  4.600047e-13  2105.9  2105.9   
                      mra29    36.0  2205.7  4.611980e-13  2205.7  2205.7   
                      mra31    41.0  1974.4  2.301983e-13  1974.4  1974.4   
                      mra32    29.0  1440.2  4.627966e-13  1440.2  1440.2   
                      mra33    40.0  2154.6  0.000000e+00  2154.6  2154.6   
                      mra34    27.0  2211.2  4.634100e-13  2211.2  2211.2   
sigmoidvenegas2offset mra26     6.0  2622.9  0.000000e+00  2622.9  2622.9   
                      mra28    27.0  2105.9  4.634100e-13  2105.9  2105.9   
                      mra29    20.0  2205.7  4.665609e-13  2205.7  2205.7   
                      mra31    20.0  1974.4  2.332805e-13  1974.4  1974.4   
                      mra32    11.0  1440.2  2.384715e-13  1440.2  1440.2   
                      mra33    20.0  2154.6  4.665609e-13  2154.6  2154.6   
                      mra34    27.0  2211.2  4.634100e-13  2211.2  2211.2   

                                                     error             \
                                 50%     75%     max count       mean   
Model                 Animal                                            
sigmoid_recruit_units mra26   2622.9  2622.9  2622.9  15.0 -30.225910   
                      mra28   2105.9  2105.9  2105.9  30.0 -17.966316   
                      mra29   2205.7  2205.7  2205.7  31.0 -35.674570   
                      mra31   1974.4  1974.4  1974.4  35.0 -35.412483   
                      mra32   1440.2  1440.2  1440.2